## A gentle 10-minute introduction to Ray AI Runitime (Ray AIR)

As part of Ray 2.0, Ray AI Runtime (AIR) is an open-source toolkit for building end-to-end ML applications. By leveraging Ray, its distributed compute capabilities, and its library ecosystem, Ray AIR brings scalability and programmability to ML platforms.

Ray AI Runtime focuses on two functional aspects:
 * It provides scalability by leveraging Ray’s distributed compute layer for ML workloads.
 * It is designed to interoperate with other systems for storage and metadata needs.

Ray AIR consists of five key components:

 * Data processing (Ray Data)
 * Model Training (Ray Train)
 * Reinforcement Learning (Ray RLlib)
 * Hyperparameter Tuning (Ray Tune)
 * Model Serving (Ray Serve).
 
 <img src = "images/ai_runtime.jpeg" width="60%" height="30%">
 
### Learning objectives:
  * Get introduced to Ray AIR as a unified toolkit to write an end-to-end ML application in a single Python script
  * Get exposed to Ray data for data ingestion
  * Use out-of-box Preprocessors
  * Load model from the best model checkpoint and use for batch inference
  * Deploy best checkpoint model and use for online inference

In [1]:
import logging, os, random, warnings
from pprint import pprint
import ray
import pandas as pd
from sklearn.model_selection import train_test_split

from ray.data.preprocessors import StandardScaler

In [2]:
warnings.filterwarnings("ignore")
os.environ["PYTHONWARNINGS"] = "ignore"

In [3]:
if ray.is_initialized:
    ray.shutdown()
context = ray.init(logging_level=logging.ERROR)
pprint(context)

RayContext(dashboard_url='127.0.0.1:8266', python_version='3.8.13', ray_version='3.0.0.dev0', ray_commit='{{RAY_COMMIT_SHA}}', address_info={'node_ip_address': '127.0.0.1', 'raylet_ip_address': '127.0.0.1', 'redis_address': None, 'object_store_address': '/tmp/ray/session_2022-07-20_10-50-36_525779_37136/sockets/plasma_store', 'raylet_socket_name': '/tmp/ray/session_2022-07-20_10-50-36_525779_37136/sockets/raylet', 'webui_url': '127.0.0.1:8266', 'session_dir': '/tmp/ray/session_2022-07-20_10-50-36_525779_37136', 'metrics_export_port': 60313, 'gcs_address': '127.0.0.1:60825', 'address': '127.0.0.1:60825', 'dashboard_agent_listen_port': 52365, 'node_id': '9636fa263e340aa24cb609b086fee61a841cb0258492f120360b1dc5'})


In [4]:
print(f"Dashboard url: http://{context.address_info['webui_url']}")

Dashboard url: http://127.0.0.1:8266


### Create Ray data from an S3 CSV datasource

In [5]:
import ray
import pandas as pd
from ray.air import train_test_split

# Split data into train and validation.
dataset = ray.data.read_csv("s3://anonymous@air-example-data/breast_cancer.csv")
train_dataset, valid_dataset = train_test_split(dataset, test_size=0.3)
test_dataset = valid_dataset.drop_columns(["target"])

Map_Batches: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 85.41it/s]


### Create Preprocessors
This preprocessor is automatically used in the training function to `fit` and `tranform` your datasets for training and validation.

In [6]:
# Create a preprocessor to scale some columns
from ray.data.preprocessors import StandardScaler

columns_to_scale = ["mean radius", "mean texture"]
preprocessor = StandardScaler(columns=columns_to_scale)

### Create Trainers
Use the Ray AIR trainer `XGBoostTrainer`. Simple steps:
 1. define the parallelism for Ray compute
 2. define the XGBoost parameters for training
 3. supply the preprocessor for fitting and transforming dataset during training and validation
 4. provide the datasets training and validation
 5. invoke `trainer.fit()` 
 
 Simple API that does a lot behind the scenes!

In [7]:
from ray.train.xgboost import XGBoostTrainer

trainer = XGBoostTrainer(
    scaling_config={
        # Number of workers to use for data parallelism.
        "num_workers": 2,
        # Whether to use GPU acceleration.
        "use_gpu": False,
    },
    label_column="target",
    num_boost_round=20,
    params={
        # XGBoost specific params
        "objective": "binary:logistic",
        "eval_metric": ["logloss", "error"],
    },
    datasets={"train": train_dataset, "valid": valid_dataset},
    preprocessor=preprocessor,
)
result = trainer.fit()
print(result.metrics)

Trial name,status,loc,iter,total time (s),train-logloss,train-error,valid-logloss
XGBoostTrainer_8445e_00000,TERMINATED,127.0.0.1:37705,20,4.69404,0.0184957,0,0.0893879


(XGBoostTrainer pid=37705) 2022-07-20 10:51:02,193	WARNING base_trainer.py:162 -- When passing `datasets` to a Trainer, it is recommended to reserve at least 20% of node CPUs for Dataset execution by setting `_max_cpu_fraction_per_node = 0.8` in the Trainer `scaling_config`. Not doing so can lead to resource contention or hangs. See https://docs.ray.io/en/master/data/key-concepts.html#example-datasets-in-tune for more info.
(XGBoostTrainer pid=37705) 2022-07-20 10:51:03,762	INFO main.py:980 -- [RayXGBoost] Created 2 new actors (2 total actors). Waiting until actors are ready for training.
(_RemoteRayXGBoostActor pid=37720) 2022-07-20 10:51:04,845	WARNING __init__.py:192 -- DeprecationWarning: `ray.worker.get_resource_ids` is a private attribute and access will be removed in a future Ray version.
(_RemoteRayXGBoostActor pid=37720)   File "/Users/jules/git-repos/ray/python/ray/_private/workers/default_worker.py", line 237, in <module>
(_RemoteRayXGBoostActor pid=37720)     ray._private.w

Result for XGBoostTrainer_8445e_00000:
  date: 2022-07-20_10-51-06
  done: false
  experiment_id: 7f1aab7f31e442649ffb9a5be3333897
  hostname: Juless-MacBook-Pro-16
  iterations_since_restore: 1
  node_ip: 127.0.0.1
  pid: 37705
  should_checkpoint: true
  time_since_restore: 4.478966951370239
  time_this_iter_s: 4.478966951370239
  time_total_s: 4.478966951370239
  timestamp: 1658339466
  timesteps_since_restore: 0
  train-error: 0.02261306532663317
  train-logloss: 0.464117960489575
  training_iteration: 1
  trial_id: 8445e_00000
  valid-error: 0.11695906432748537
  valid-logloss: 0.5025240946234318
  warmup_time: 0.0026171207427978516
  


(XGBoostTrainer pid=37705) 2022-07-20 10:51:06,893	INFO main.py:1516 -- [RayXGBoost] Finished XGBoost training on training data with total N=398 in 3.14 seconds (1.93 pure XGBoost training time).


Result for XGBoostTrainer_8445e_00000:
  date: 2022-07-20_10-51-06
  done: true
  experiment_id: 7f1aab7f31e442649ffb9a5be3333897
  experiment_tag: '0'
  hostname: Juless-MacBook-Pro-16
  iterations_since_restore: 20
  node_ip: 127.0.0.1
  pid: 37705
  should_checkpoint: true
  time_since_restore: 4.694040060043335
  time_this_iter_s: 0.006032228469848633
  time_total_s: 4.694040060043335
  timestamp: 1658339466
  timesteps_since_restore: 0
  train-error: 0.0
  train-logloss: 0.01849572773292735
  training_iteration: 20
  trial_id: 8445e_00000
  valid-error: 0.04093567251461988
  valid-logloss: 0.08938791319913073
  warmup_time: 0.0026171207427978516
  
{'train-logloss': 0.01849572773292735, 'train-error': 0.0, 'valid-logloss': 0.08938791319913073, 'valid-error': 0.04093567251461988, 'time_this_iter_s': 0.006032228469848633, 'should_checkpoint': True, 'done': True, 'timesteps_total': None, 'episodes_total': None, 'training_iteration': 20, 'trial_id': '8445e_00000', 'experiment_id': '7f

### Create Tuner for hyperparameter search

What if you want to do hyperparameter optimization during training and use the best config for the model? 
Well, you can then use Tuner and supply your training function, Trainer, as part of the argument, along with other Tuner
configuration. 

Again, simple steps:
 1. define your hyperparameter space
 2. define `TuneConfig` for number of trials and parallelism 
 3. invoke `tuner.fit()`

In [8]:
from ray import tune

param_space = {"params": {"max_depth": tune.randint(1, 9)}}
metric = "train-logloss"

In [9]:
from ray.tune.tuner import Tuner, TuneConfig
from ray.air.config import RunConfig

tuner = Tuner(
    trainer,
    param_space=param_space,
    tune_config=TuneConfig(num_samples=5, metric=metric, mode="min"),
)
# Execute tuning.
result_grid = tuner.fit()

# Fetch the best result with its best hyperparameter config 
best_result = result_grid.get_best_result()
print("Best Result:", best_result)

Trial name,status,loc,params/max_depth,iter,total time (s),train-logloss,train-error,valid-logloss
XGBoostTrainer_8c9fe_00000,TERMINATED,127.0.0.1:37732,6,20,2.59019,0.0184957,0,0.0893879
XGBoostTrainer_8c9fe_00001,TERMINATED,127.0.0.1:37735,3,20,4.71449,0.0215151,0,0.0765915
XGBoostTrainer_8c9fe_00002,TERMINATED,127.0.0.1:37736,6,20,4.71488,0.0184957,0,0.0893879
XGBoostTrainer_8c9fe_00003,TERMINATED,127.0.0.1:37797,5,20,4.82036,0.0184163,0,0.105782
XGBoostTrainer_8c9fe_00004,TERMINATED,127.0.0.1:37807,3,20,3.04534,0.0215151,0,0.0765915


(XGBoostTrainer pid=37732) 2022-07-20 10:51:16,014	WARNING base_trainer.py:162 -- When passing `datasets` to a Trainer, it is recommended to reserve at least 20% of node CPUs for Dataset execution by setting `_max_cpu_fraction_per_node = 0.8` in the Trainer `scaling_config`. Not doing so can lead to resource contention or hangs. See https://docs.ray.io/en/master/data/key-concepts.html#example-datasets-in-tune for more info.
(XGBoostTrainer pid=37732) 2022-07-20 10:51:16,144	INFO main.py:980 -- [RayXGBoost] Created 2 new actors (2 total actors). Waiting until actors are ready for training.
(XGBoostTrainer pid=37736) 2022-07-20 10:51:17,188	WARNING base_trainer.py:162 -- When passing `datasets` to a Trainer, it is recommended to reserve at least 20% of node CPUs for Dataset execution by setting `_max_cpu_fraction_per_node = 0.8` in the Trainer `scaling_config`. Not doing so can lead to resource contention or hangs. See https://docs.ray.io/en/master/data/key-concepts.html#example-datasets

Result for XGBoostTrainer_8c9fe_00000:
  date: 2022-07-20_10-51-18
  done: false
  experiment_id: ea59ac27b07a45e6a95ca69f0922911b
  hostname: Juless-MacBook-Pro-16
  iterations_since_restore: 1
  node_ip: 127.0.0.1
  pid: 37732
  should_checkpoint: true
  time_since_restore: 2.3179068565368652
  time_this_iter_s: 2.3179068565368652
  time_total_s: 2.3179068565368652
  timestamp: 1658339478
  timesteps_since_restore: 0
  train-error: 0.02261306532663317
  train-logloss: 0.464117960489575
  training_iteration: 1
  trial_id: 8c9fe_00000
  valid-error: 0.11695906432748537
  valid-logloss: 0.5025240946234318
  warmup_time: 0.002615213394165039
  


(_RemoteRayXGBoostActor pid=37758) 2022-07-20 10:51:18,462	WARNING __init__.py:192 -- DeprecationWarning: `ray.worker.get_resource_ids` is a private attribute and access will be removed in a future Ray version.
(_RemoteRayXGBoostActor pid=37758)   File "/Users/jules/git-repos/ray/python/ray/_private/workers/default_worker.py", line 237, in <module>
(_RemoteRayXGBoostActor pid=37758)     ray._private.worker.global_worker.main_loop()
(_RemoteRayXGBoostActor pid=37758)   File "/Users/jules/git-repos/ray/python/ray/_private/worker.py", line 754, in main_loop
(_RemoteRayXGBoostActor pid=37758)     self.core_worker.run_task_loop()
(_RemoteRayXGBoostActor pid=37758)   File "/Users/jules/git-repos/ray/python/ray/_private/function_manager.py", line 674, in actor_method_executor
(_RemoteRayXGBoostActor pid=37758)     return method(__ray_actor, *args, **kwargs)
(_RemoteRayXGBoostActor pid=37758)   File "/Users/jules/git-repos/ray/python/ray/util/tracing/tracing_helper.py", line 466, in _resume_sp

Result for XGBoostTrainer_8c9fe_00000:
  date: 2022-07-20_10-51-18
  done: true
  experiment_id: ea59ac27b07a45e6a95ca69f0922911b
  experiment_tag: 0_max_depth=6
  hostname: Juless-MacBook-Pro-16
  iterations_since_restore: 20
  node_ip: 127.0.0.1
  pid: 37732
  should_checkpoint: true
  time_since_restore: 2.590191125869751
  time_this_iter_s: 0.006448030471801758
  time_total_s: 2.590191125869751
  timestamp: 1658339478
  timesteps_since_restore: 0
  train-error: 0.0
  train-logloss: 0.01849572773292735
  training_iteration: 20
  trial_id: 8c9fe_00000
  valid-error: 0.04093567251461988
  valid-logloss: 0.08938791319913073
  warmup_time: 0.002615213394165039
  


...

Result for XGBoostTrainer_8c9fe_00003:
  date: 2022-07-20_10-51-26
  done: false
  experiment_id: d4fe8d1c1b3143fcbbb72c60b7c3af14
  hostname: Juless-MacBook-Pro-16
  iterations_since_restore: 1
  node_ip: 127.0.0.1
  pid: 37797
  should_checkpoint: true
  time_since_restore: 4.623799800872803
  time_this_iter_s: 4.623799800872803
  time_total_s: 4.623799800872803
  timestamp: 1658339486
  timesteps_since_restore: 0
  train-error: 0.02261306532663317
  train-logloss: 0.465611254524945
  training_iteration: 1
  trial_id: 8c9fe_00003
  valid-error: 0.0935672514619883
  valid-logloss: 0.5058815336366843
  warmup_time: 0.0027060508728027344
  


(XGBoostTrainer pid=37797) 2022-07-20 10:51:26,538	INFO main.py:1516 -- [RayXGBoost] Finished XGBoost training on training data with total N=398 in 3.17 seconds (1.98 pure XGBoost training time).


Result for XGBoostTrainer_8c9fe_00004:
  date: 2022-07-20_10-51-26
  done: false
  experiment_id: 515f978fe4ab42639814cb09799a2311
  hostname: Juless-MacBook-Pro-16
  iterations_since_restore: 1
  node_ip: 127.0.0.1
  pid: 37807
  should_checkpoint: true
  time_since_restore: 2.9219861030578613
  time_this_iter_s: 2.9219861030578613
  time_total_s: 2.9219861030578613
  timestamp: 1658339486
  timesteps_since_restore: 0
  train-error: 0.03517587939698492
  train-logloss: 0.47431553248784053
  training_iteration: 1
  trial_id: 8c9fe_00004
  valid-error: 0.09941520467836257
  valid-logloss: 0.5004687657830311
  warmup_time: 0.002939939498901367
  


(XGBoostTrainer pid=37807) 2022-07-20 10:51:26,823	INFO main.py:1516 -- [RayXGBoost] Finished XGBoost training on training data with total N=398 in 3.00 seconds (1.80 pure XGBoost training time).


Result for XGBoostTrainer_8c9fe_00003:
  date: 2022-07-20_10-51-26
  done: true
  experiment_id: d4fe8d1c1b3143fcbbb72c60b7c3af14
  experiment_tag: 3_max_depth=5
  hostname: Juless-MacBook-Pro-16
  iterations_since_restore: 20
  node_ip: 127.0.0.1
  pid: 37797
  should_checkpoint: true
  time_since_restore: 4.8203630447387695
  time_this_iter_s: 0.005700111389160156
  time_total_s: 4.8203630447387695
  timestamp: 1658339486
  timesteps_since_restore: 0
  train-error: 0.0
  train-logloss: 0.01841634292981527
  training_iteration: 20
  trial_id: 8c9fe_00003
  valid-error: 0.05263157894736842
  valid-logloss: 0.10578184703239703
  warmup_time: 0.0027060508728027344
  
Result for XGBoostTrainer_8c9fe_00004:
  date: 2022-07-20_10-51-26
  done: true
  experiment_id: 515f978fe4ab42639814cb09799a2311
  experiment_tag: 4_max_depth=3
  hostname: Juless-MacBook-Pro-16
  iterations_since_restore: 20
  node_ip: 127.0.0.1
  pid: 37807
  should_checkpoint: true
  time_since_restore: 3.045338153839111

### Create a `BatchPreditor` for batch prediction
Once you have trained and tuned your model, create a batch predictor from best model using the `best_result.checkpoint` and do batch inference. 

In [10]:
from ray.train.batch_predictor import BatchPredictor
from ray.train.xgboost import XGBoostPredictor

batch_predictor = BatchPredictor.from_checkpoint(best_result.checkpoint, XGBoostPredictor)

predicted_probabilities = batch_predictor.predict(test_dataset)
print("PREDICTED PROBABILITIES")
predicted_probabilities.show()

Map Progress (1 actors 1 pending): 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.38it/s]

PREDICTED PROBABILITIES
{'predictions': 0.9960426092147827}
{'predictions': 0.9957077503204346}
{'predictions': 0.0034389763604849577}
{'predictions': 0.9962536096572876}
{'predictions': 0.9968380928039551}
{'predictions': 0.9957551956176758}
{'predictions': 0.9920042157173157}
{'predictions': 0.994161069393158}
{'predictions': 0.2891101539134979}
{'predictions': 0.974367082118988}
{'predictions': 0.0034389763604849577}
{'predictions': 0.9959942102432251}
{'predictions': 0.9474029541015625}
{'predictions': 0.9923243522644043}
{'predictions': 0.9941523671150208}
{'predictions': 0.1239369809627533}
{'predictions': 0.5043733716011047}
{'predictions': 0.9935414791107178}
{'predictions': 0.9832899570465088}
{'predictions': 0.0034389763604849577}


### Create `PredictorDeployment` for Online Inference

Deploy the best model as an inference service by using Ray Serve and the `PredictorDeployment` class.

In [11]:
from ray import serve
from fastapi import Request
from ray.serve import PredictorDeployment
from ray.serve.http_adapters import json_request


async def adapter(request: Request):
    content = await request.json()
    print(content)
    return pd.DataFrame.from_dict(content)


serve.start(detached=True)
deployment = PredictorDeployment.options(name="XGBoostService")

deployment.deploy(
    XGBoostPredictor, best_result.checkpoint, batching_params=False, http_adapter=adapter
)

print(deployment.url)

(ServeController pid=37930) INFO 2022-07-20 10:51:47,280 controller 37930 checkpoint_path.py:17 - Using RayInternalKVStore for controller checkpoint and recovery.
(ServeController pid=37930) INFO 2022-07-20 10:51:47,300 controller 37930 http_state.py:123 - Starting HTTP proxy with name 'SERVE_CONTROLLER_ACTOR:SERVE_PROXY_ACTOR-9636fa263e340aa24cb609b086fee61a841cb0258492f120360b1dc5' on node '9636fa263e340aa24cb609b086fee61a841cb0258492f120360b1dc5' listening on '127.0.0.1:8000'
(HTTPProxyActor pid=37932) INFO:     Started server process [37932]
(ServeController pid=37930) INFO 2022-07-20 10:51:47,921 controller 37930 deployment_state.py:1280 - Adding 1 replicas to deployment 'XGBoostService'.


http://127.0.0.1:8000/XGBoostService


After deploying the service, you can send requests to it.

In [12]:
import requests

sample_input = test_dataset.take(1)
sample_input = dict(sample_input[0])

output = requests.post(deployment.url, json=[sample_input]).json()
print(output)

[{'predictions': 0.9960426092147827}]
(ServeReplica:XGBoostService pid=37934) [{'mean radius': 11.06, 'mean texture': 14.83, 'mean perimeter': 70.31, 'mean area': 378.2, 'mean smoothness': 0.07741, 'mean compactness': 0.04768, 'mean concavity': 0.02712, 'mean concave points': 0.007246, 'mean symmetry': 0.1535, 'mean fractal dimension': 0.06214, 'radius error': 0.1855, 'texture error': 0.6881, 'perimeter error': 1.263, 'area error': 12.98, 'smoothness error': 0.004259, 'compactness error': 0.01469, 'concavity error': 0.0194, 'concave points error': 0.004168, 'symmetry error': 0.01191, 'fractal dimension error': 0.003537, 'worst radius': 12.68, 'worst texture': 20.35, 'worst perimeter': 80.79, 'worst area': 496.7, 'worst smoothness': 0.112, 'worst compactness': 0.1879, 'worst concavity': 0.2079, 'worst concave points': 0.05556, 'worst symmetry': 0.259, 'worst fractal dimension': 0.09158}]


(HTTPProxyActor pid=37932) INFO 2022-07-20 10:51:54,106 http_proxy 127.0.0.1 http_proxy.py:315 - POST /XGBoostService 307 5.4ms
(HTTPProxyActor pid=37932) INFO 2022-07-20 10:51:54,114 http_proxy 127.0.0.1 http_proxy.py:315 - POST /XGBoostService 200 6.6ms
(ServeReplica:XGBoostService pid=37934) INFO 2022-07-20 10:51:54,104 XGBoostService XGBoostService#kLSUrh replica.py:478 - HANDLE __call__ OK 0.3ms
(ServeReplica:XGBoostService pid=37934) INFO 2022-07-20 10:51:54,114 XGBoostService XGBoostService#kLSUrh replica.py:478 - HANDLE __call__ OK 4.7ms


In [13]:
ray.shutdown()

### Homework

1. Have a go at Ray AIR examples in the documentation.